#Osservazioni codice#



*   **check_time**: devo usare anche il domain del sito per forza?
*   **fetch**: devo usare un if se è la prima volta che visito l'url cosi richiamo il robots.txt? quando devo controllare la sitema etc.?


#Librerie#

In [ ]:
#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito giocattolo ha tutto
start_url = "https://nonciclopedia.org/wiki/Wikipedia"

In [ ]:
import pandas as pd
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)
import rfc3986 # per la normalizzazione degli urls


#Fetcher#
gestione richieste e download pagine web

In [ ]:
def chech_roobots(url,useragent="*"):
  '''
  input: useragent (default: *, generico), url sito da analizzare
  output: check,path_disallow,crawl_delay,request_rate,sitemap

  controlla il file robot.txt associato all'url
  '''

  rfl = urllib.robotparser.RobotFileParser()
  # aggiungo all'url selezionato l'estensione corretta per leggere il file robots.txt
  new_url = url + "/robots.txt"

  rfl.set_url(new_url)
  #legge il file robots.txt e controlla se lo useragent puo effettuare il fetch dei dati
  rfl.read()
  check = rfl.can_fetch(useragent,new_url)
  if check:
    print("{} il sito puo effettuare il fetch dei dati".format(check))
  else:
    print("il sito non puo effettuare il fetch dei dati")

  # salva le path non accessibili (Disallow:...)
  path_disallow = rfl.parse("Disallow")
  print("path non accessibili: {}".format(path_disallow))

  # salva il craw_delay: intervallo temporale tra richieste dello stesso useragent
  crawl_delay = rfl.crawl_delay(useragent)
  print("crawl delay: {}".format(crawl_delay))

  # salva il request_rate: numero di richieste ogni tot secondi (tuple (r,s))
  request_rate = request_rate(useragent)
  print("request rate: {}".format(request_rate))

  # salva la sitemap
  sitemap = rfl.site_maps()
  print("sitemap: {}".format(sitemap))



  return check,path_disallow,crawl_delay,request_rate,sitemap

In [ ]:
class UserAgentPolicy
  '''
    classe per identificare gli useragent e salvare di volta in volta le informazioni necessarie per gestire le richieste
  '''
  def __init__(self, name, base_url, path_dissalow, crawl_delay, request_rate, n_request,last_access, header):
        self.base_url = base_url
        self.name = name
        self.path_dissalow = path_dissalow
        self.crawl_delay = crawl_delay
        self.request_rate = request_rate
        self.n_request = n_request
        self.last_access = last_access
        self.header = header

In [ ]:
#Useragents di prova#
useragent_dict = {"Googlebot": UserAgentPolicy("Googlebot",None, None, None, None, None, None,'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'),
                  "Bingbot": UserAgentPolicy("Bingbot",None, None, None, None, None, None,'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)'),
                  "Slurp": UserAgentPolicy("Slurp",None, None, None, None, None, None, 'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)'),
                  "DuckDuckbot": UserAgentPolicy("DuckDuckbot",None, None, None, None, None, None,'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)' ),
                  "Yandex": UserAgentPolicy("Yandex",None, None, None, None, None, None,'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)' ),
                  "*": UserAgentPolicy("*",None, None, None, None, None, None,'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)' )
                  }
default_agent = useragent_dict["*"]

# per fare test e simulare il comportamento di più useragent (non so se metterli nelle classi)
useragent_dict_header = {'Googlebot':'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)',
                    'Bingbot':'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)',
                    'Slurp':'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)',
                    'DuckDuckbot':'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)',
                    'Yandex':'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)',
                    '*': 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'} #---> non so se devo mettere solo '*' anche su questo header (generato con ChatGPT)

In [ ]:
def check_time(useragent=default_agent):
  '''
    controlla che lo useragent rispetti il tempo di craw-delay e il request_rate
  '''

  # Se entrambi definiti, scegli il valore più restrittivo (conversione in ns)
  if useragent.request_rate and useragent.crawl_delay:
     request_delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
     crawl_delay = useragent.crawl_delay * 1e9
     delay = max(request_delay, crawl_delay)

    elif useragent.crawl_delay:
        delay = useragent.crawl_delay * 1e9

    elif useragent.request_rate:
        delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]

    now = time.monotonic_ns()
    wait = max(0, (useragent.last_access + delay) - now)

    if wait > 0:
        time.sleep(wait / 1e9)  # converto da ns a s limiti sleep

    # aggiorno a prescindere dal succeso della connesione il last_access così da rispettare di sicuro la policy
    useragent.last_access = time.monotonic_ns()


In [ ]:
def fetch(url,useragent=default_agent):
  '''
    contatta la pagina e gestisce le richieste da parte dello useragent

  '''

  # controllo che lo user non esegua troppe richieste allo stesso DNS
  check_time(useragent)

  # costuisco l'header specifico per lo useragent richiesto(usato per test, non semrpe necessario)
  headers = {
        'User-Agent': useragent.header
    }
  # contatto il sito
  try:
     response = requests.get(url, headers=headers, timeout=10)
     if response.status_code == 200:
       print("Pagina recuperata con successo:{}",.format(url))
       html = response.text
       return html
     else:
      print("Errore nella ricezione del sito:{}".format(url))
      return None

  # per gestire tutti gli altri errori
  except requests.RequestException as e:
        print("Eccezione durante la richiesta:", e)
        return None

# Parser#
analisi contenuto della pagina, estrazione testo e link e normalizzazione link trovati

In [ ]:
def normalize_url(url):
'''
   Normalizza l'URL secondo lo standard RFC3986 e restituisce una stringa
'''

    try:
        uri = rfc3986.uri_reference(url).normalize()
        return uri.unsplit()

    except Exception as e:
        print(f"Errore nella normalizzazione URL: {url} – {e}")
        return url

In [ ]:
def parse_sitemap(sitemap_list):
    '''
    Prende in input una lista di sitemap URLs e restituisce un DataFrame
    con: URL, priorità e frequenza di aggiornamento.
    '''
    sitemap_all = pd.DataFrame(columns=['url','priority','update'])

    # Standard XML Namespace ufficiale del protocollo Sitemap, serve per identificare il vocabolario e la struttura usata nel file xml
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    def parse_single_sitemap(sitemap_url):

        response = requests.get(sitemap_url)
        if response.status_code != 200:
           print("Errore nel recupero:", sitemap_url)
           content_type = response.headers.get("Content-Type", "")
           return content_type

    # Sitemap XML
    if "xml" in content_type:

        # Controllo che la sitemap risponda (da aggiungere la possibilità di decompressione se necessaria)
        root = ET.fromstring(response.content)

        # Se è una sitemap index (contiene altre sitemap)
        if root.tag.endswith('index'):
            sitemap_urls = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
            all_entries = []
            for sub_url in sitemap_urls:
                all_entries.extend(parse_single_sitemap(sub_url))
            return all_entries

        # Se è una sitemap normale (contiene solo URL)
        elif root.tag.endswith('urlset'):
            entries = []
            for url in root.findall('ns:url', namespace):
                loc = url.find('ns:loc', namespace)
                priority = url.find('ns:priority', namespace)
                changefreq = url.find('ns:changefreq', namespace)

                entries.append({
                    'url': loc.text if loc is not None else '',
                    'priority': priority.text if priority is not None else None,
                    'update': changefreq.text if changefreq is not None else None
                })
            return entries

    # Sitemap HTML
    elif "html" in content_type:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.find_all('a', href=True)
        entries = []
        for a in links:
            url = normalize_url(a['href'])
            entries.append({'url': url, 'priority': None, 'update': None})
        return entries

    else:
        print(f"Formato non riconosciuto per {sitemap_url}")
        return []


    # Analizza ogni sitemap passata
    all_data = []

    if sitemap_list is None:
        print("Nessuna sitemap presente")
        return sitemap_all

    for sitemap_url in sitemap_list:
        if sitemap_url is None:
            print("Nessuna sitemap presente")
            continue
        all_data.extend(parse_single_sitemap(sitemap_url))

    # Normalizzo gli url
    for entry in all_data:
        entry['url'] = normalize_url(entry['url'])

    # Crea DataFrame finale
    sitemap_all = pd.DataFrame(all_data)

    # Elimino i duplicati
    sitemap_all = sitemap_all.drop_duplicates()

    return sitemap_all


In [ ]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup

def parse_sitemap(sitemap_list):
    '''
    Prende in input una lista di sitemap URLs e restituisce un DataFrame
    con: URL, priorità e frequenza di aggiornamento.
    '''
    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
    all_entries = []

    if not sitemap_list:
        print("Nessuna sitemap fornita.")
        return pd.DataFrame(columns=['url', 'priority', 'update'])

    def parse_single_sitemap(sitemap_url):
        try:
            response = requests.get(sitemap_url, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Errore durante il recupero di {sitemap_url}: {e}")
            return []

        content_type = response.headers.get("Content-Type", "").lower()

        # Sitemap XML
        if "xml" in content_type:
            try:
                root = ET.fromstring(response.content)
            except ET.ParseError as e:
                print(f"Errore nel parsing XML: {e}")
                return []

            # Sitemap index
            if root.tag.endswith('index'):
                sitemap_urls = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
                entries = []
                for sub_url in sitemap_urls:
                    entries.extend(parse_single_sitemap(sub_url))
                return entries

            # Sitemap semplice
            elif root.tag.endswith('urlset'):
                return [{
                    'url': url.findtext('ns:loc', default='', namespaces=namespace),
                    'priority': url.findtext('ns:priority', default=None, namespaces=namespace),
                    'update': url.findtext('ns:changefreq', default=None, namespaces=namespace)
                } for url in root.findall('ns:url', namespace)]

        # Sitemap HTML
        elif "html" in content_type:
            soup = BeautifulSoup(response.content, "html.parser")
            return [{
                'url': normalize_url(a['href']),
                'priority': None,
                'update': None
            } for a in soup.find_all('a', href=True)]

        else:
            print(f"Formato non riconosciuto per {sitemap_url} ({content_type})")
            return []

    # Estrai tutte le voci da ogni sitemap
    for sitemap_url in sitemap_list:
        if sitemap_url:
            entries = parse_single_sitemap(sitemap_url)
            for entry in entries:
                entry['url'] = normalize_url(entry['url'])  # Normalizzazione
            all_entries.extend(entries)

    df = pd.DataFrame(all_entries, columns=['url', 'priority', 'update']).drop_duplicates()

    return df


In [ ]:
def parse_page_url(html,sitemaps_urls,useragent=defaullt_agent):
  '''
    dato una pagina html di input cerco gli urls:
    - se sono in una directory privata li scarto
    - se gia presenti nella sitemap, sono già inidicizzate e non salvo i link (duplicati)
    - se non presenti salvo gli url permessi dal file robots.txt
  '''

  #parsing pagina html
  soup = BeautifulSoup(html, 'html.parser')

  urls = []
  # Elementi con attributo href
  for tag in soup.find_all(href=True):
      urls.append(tag['href'])

  # Elementi con attributo src
  for tag in soup.find_all(src=True):
      urls.append(tag['src'])

  # controlla se i nuovi url sono veramente accessibili per il web scraping (no in disallow)
  if useragent.path_disallow != None:
    for url in urls:
      for path in useragent.path_disallow:
        if path in url:
         urls.remove(url)

  # normalizzo gli urls
    urls = [normalize_url(url) for url in urls]

  # elimino i duplicati
    urls = list(set(urls))


  #controlla se ci sono duplicati nella sitemap se c'è
  if sitemaps_urls != None:
    def_urls = sitemaps_urls
    for url in urls:
      if url not in def_urls:
         def_urls.append(url)
  else:
    def_urls = urls

  return def_urls



In [ ]:
def parse_page_tags(html,tag_type, attrs={}):
 '''
    accetta in input una pagina html, il tipo di taga da selezioanre ed eventuali attributi se necessari
 '''
    # scelgo il parser
    soup = BeautifulSoup(response.content, "html.parser")
    tags = soup.find_all(tag_type, attrs=attrs)

    return [str(tag) for tag in tags]